In [6]:
import sys
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import models
from PIL import Image
import matplotlib.pyplot as plt
from tabulate import tabulate
from tqdm import tqdm

# Assuming the path to your RISE-SFL folder is set up correctly
rise_sfl_path = r"C:\Users\eagua\OneDrive\Documents\GitHub\6CCS3COV-7CCSMCVI-Computer-Vision\RISEX"
sys.path.append(rise_sfl_path)
# Assuming these imports work as in your original script
from explanations import RISE
from utils import read_tensor, get_class_name, tensor_imshow

rise_sfl_path = r"C:\Users\eagua\OneDrive\Documents\GitHub\6CCS3COV-7CCSMCVI-Computer-Vision\metrics-saliency-maps"  # Replace with the actual path to your RISE-SFL folder
sys.path.append(rise_sfl_path)
#saliency_maps = extract_specific_saliency_maps(pixel_datasets[0])
from saliency_maps_metrics.multi_step_metrics import Deletion, Insertion, compute_correlation, compute_auc_metric
from saliency_maps_metrics.data_replace import select_data_replace_method


# Make sure this import works with your file structure

print("Script started")

# Set up CUDA
import torch.backends.cudnn as cudnn
cudnn.benchmark = True

# Load the model
model = models.resnet50(True)
model = nn.Sequential(model, nn.Softmax(dim=1))
model = model.eval()
model = model.cuda()

for p in model.parameters():
    p.requires_grad = False

print("Model loaded")

N= 200
s = 8
p1 = 0.2
# Initialize RISE explainer
explainer = RISE(model, (224, 224))

explainer.load_masks(N,p1,'mask.npy')  # This should load masks with shape (19, 10, 1, 224, 224)

print("RISE explainer initialized")

# Set up metrics
deletion_metric = Deletion(data_replace_method="black", bound_max_step=True, batch_size=1, max_step_nb=14*14, cumulative=True)
insertion_metric = Insertion(data_replace_method="blur", bound_max_step=True, batch_size=1, max_step_nb=14*14, cumulative=True)

# Process all images in the folder
image_folder = r'C:\Users\eagua\OneDrive\Documents\GitHub\6CCS3COV-7CCSMCVI-Computer-Vision\causal-explainer\data_benchmark'

print(f"Checking image folder: {image_folder}")
print(f"Folder exists: {os.path.exists(image_folder)}")
print(f"Folder contents: {os.listdir(image_folder)}")

image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

print(f"Found {len(image_files)} images in the folder")
print(f"Image files: {image_files}")

# Function to calculate sparsity
def calculate_sparsity(saliency_map):
    max_value = np.max(saliency_map)
    mean_value = np.mean(saliency_map)
    return max_value / mean_value if mean_value != 0 else float('inf')

# Initialize dictionary to store cumulative results
cumulative_results = {
    "RISE": {"DAUC": 0, "DC": 0, "IAUC": 0, "IC": 0, "Sparsity": 0}
}

# Process each image
for image_index, image_file in enumerate(tqdm(image_files, desc="Processing images")):
    img_path = os.path.join(image_folder, image_file)
    img = read_tensor(img_path).cuda()

    # Get prediction
    with torch.no_grad():
        output = model(img)
    target_class = output.argmax().item()

    # Generate saliency map
    sal_og = explainer(img, image_index=image_index)[target_class].cpu().numpy()

    # Calculate metrics
    saliency_map_tensor = torch.from_numpy(sal_og).float().unsqueeze(0).unsqueeze(0).cuda()

    # Compute Deletion metrics
    deletion_result = deletion_metric(model, img, saliency_map_tensor, class_to_explain_list=[target_class])
    cumulative_results["RISE"]["DAUC"] += deletion_result["dauc"]
    cumulative_results["RISE"]["DC"] += deletion_result["dc"]

    # Compute Insertion metrics
    insertion_result = insertion_metric(model, img, saliency_map_tensor, class_to_explain_list=[target_class])
    cumulative_results["RISE"]["IAUC"] += insertion_result["iauc"]
    cumulative_results["RISE"]["IC"] += insertion_result["ic"]

    # Calculate sparsity
    cumulative_results["RISE"]["Sparsity"] += calculate_sparsity(sal_og)

    # Visualize and save saliency map
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    tensor_imshow(img[0].cpu())
    plt.title(f"Original - {get_class_name(target_class)}")
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    tensor_imshow(img[0].cpu())
    plt.imshow(sal_og, cmap='jet', alpha=0.5)
    plt.title(f"Saliency Map (Image {image_index + 1})")
    plt.axis('off')
    
    plt.savefig(f"saliency_{image_file}_image{image_index + 1}.png")
    plt.close()

# Calculate averages
num_images = len(image_files)
results = []
for name, metrics in cumulative_results.items():
    avg_metrics = [name] + [metrics[key] / num_images for key in ["DAUC", "DC", "IAUC", "IC", "Sparsity"]]
    results.append(avg_metrics)

# Create a table using tabulate
headers = ["Saliency Map", "Avg DAUC", "Avg DC", "Avg IAUC", "Avg IC", "Avg Sparsity"]
table = tabulate(results, headers=headers, tablefmt="grid", floatfmt=".4f")

# Print the table
print(f"\nAverage Metrics for {num_images} images:")
print(table)

print("Script completed")

Script started
Model loaded


TypeError: expected str, bytes or os.PathLike object, not int

RISE ORIGINAL

In [ ]:
import sys
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import models
from PIL import Image
import matplotlib.pyplot as plt
from tabulate import tabulate
from tqdm import tqdm



# Set up the path to your RISE-SFL folder
rise_sfl_path = r"C:\Users\eagua\OneDrive\Documents\GitHub\6CCS3COV-7CCSMCVI-Computer-Vision\Causal-AI"
sys.path.append(rise_sfl_path)

# Assuming these imports work as in your original script
from explanations import RISE
from utils import read_tensor, get_class_name, tensor_imshow
print("Script started")
import torch.backends.cudnn as cudnn
cudnn.benchmark = True
model = models.resnet50(True)
model = nn.Sequential(model, nn.Softmax(dim=1))
model = model.eval()
model = model.cuda()

for p in model.parameters():
    p.requires_grad = False


print("Model loaded")

# Initialize RISE explainer
explainer = RISE(model, (224, 224))
N, s, p1 = 200, 8, 0.2
explainer.generate_masks(N, s, p1)
explainer.load_masks('masks.npy',p1,s)

print("RISE explainer initialized")

# Process all images in the folder
image_folder = r'C:\Users\eagua\OneDrive\Desktop\data_benchmark'

print(f"Checking image folder: {image_folder}")
print(f"Folder exists: {os.path.exists(image_folder)}")
print(f"Folder contents: {os.listdir(image_folder)}")

image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

print(f"Found {len(image_files)} images in the folder")
print(f"Image files: {image_files}")

# Function to calculate sparsity
def calculate_sparsity(saliency_map): 
    max_value = np.max(saliency_map)
    mean_value = np.mean(saliency_map)
    return max_value / mean_value if mean_value != 0 else float('inf')

rise_sfl_path = r"C:\Users\eagua\OneDrive\Documents\GitHub\6CCS3COV-7CCSMCVI-Computer-Vision\metrics-saliency-maps"  # Replace with the actual path to your RISE-SFL folder
sys.path.append(rise_sfl_path)
#saliency_maps = extract_specific_saliency_maps(pixel_datasets[0])

from saliency_maps_metrics.multi_step_metrics import Deletion, Insertion, compute_correlation, compute_auc_metric
from saliency_maps_metrics.data_replace import select_data_replace_method

deletion_metric = Deletion(data_replace_method="black", bound_max_step=True, batch_size=1, max_step_nb=14*14, cumulative=True)
insertion_metric = Insertion(data_replace_method="blur", bound_max_step=True, batch_size=1, max_step_nb=14*14, cumulative=True)


# Initialize dictionary to store cumulative results
cumulative_results = {
    "RISE": {"DAUC": 0, "DC": 0, "IAUC": 0, "IC": 0, "Sparsity": 0}
}

# Process each image
for image_file in tqdm(image_files, desc="Processing images"):
    img_path = os.path.join(image_folder, image_file)
    img = read_tensor(img_path).cuda()

    # Get prediction
    with torch.no_grad():
        output = model(img)
    target_class = output.argmax().item()

    # Generate saliency map
    sal_og = explainer(img)[target_class].cpu().numpy()

    # Calculate metrics
    saliency_map_tensor = torch.from_numpy(sal_og).float().unsqueeze(0).unsqueeze(0).cuda()

    # Compute Deletion metrics
    deletion_result = deletion_metric(model, img, saliency_map_tensor, class_to_explain_list=[target_class])
    cumulative_results["RISE"]["DAUC"] += deletion_result["dauc"]
    cumulative_results["RISE"]["DC"] += deletion_result["dc"]

    # Compute Insertion metrics
    insertion_result = insertion_metric(model, img, saliency_map_tensor, class_to_explain_list=[target_class])
    cumulative_results["RISE"]["IAUC"] += insertion_result["iauc"]
    cumulative_results["RISE"]["IC"] += insertion_result["ic"]

    # Calculate sparsity
    cumulative_results["RISE"]["Sparsity"] += calculate_sparsity(sal_og)

    # Visualize and save saliency map
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    tensor_imshow(img[0].cpu())
    plt.title(f"Original - {get_class_name(target_class)}")
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    tensor_imshow(img[0].cpu())
    plt.imshow(sal_og, cmap='jet', alpha=0.5)
    plt.title("Saliency Map")
    plt.axis('off')
    
    plt.savefig(f"saliency_{image_file}.png")
    plt.close()

# Calculate averages
num_images = len(image_files)
results = []
for name, metrics in cumulative_results.items():
    avg_metrics = [name] + [metrics[key] / num_images for key in ["DAUC", "DC", "IAUC", "IC", "Sparsity"]]
    results.append(avg_metrics)

# Create a table using tabulate
headers = ["Saliency Map", "Avg DAUC", "Avg DC", "Avg IAUC", "Avg IC", "Avg Sparsity"]
table = tabulate(results, headers=headers, tablefmt="grid", floatfmt=".4f")

# Print the table
print(f"\nAverage Metrics for {num_images} images:")
print(table)

print("Script completed")

In [4]:
import sys
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import models
from PIL import Image
import matplotlib.pyplot as plt
from tabulate import tabulate
from tqdm import tqdm

# Set up the path to your RISE-SFL folder
rise_sfl_path = r"C:\Users\eagua\OneDrive\Documents\GitHub\6CCS3COV-7CCSMCVI-Computer-Vision\Causal-AI"
sys.path.append(rise_sfl_path)

from explanations import RISE
from utils import read_tensor, get_class_name, tensor_imshow

metrics_path = r"C:\Users\eagua\OneDrive\Desktop\data_benchmark"
sys.path.append(metrics_path)
from saliency_maps_metrics.multi_step_metrics import Deletion, Insertion, compute_correlation, compute_auc_metric
from saliency_maps_metrics.data_replace import select_data_replace_method

print("Script started")

# Set up CUDA
import torch.backends.cudnn as cudnn
cudnn.benchmark = True

def analyze_saliency_maps(image_folder, analysis=False):
    # Load the model
    model = models.resnet50(True)
    model = nn.Sequential(model, nn.Softmax(dim=1))
    model = model.eval()
    model = model.cuda()

    for p in model.parameters():
        p.requires_grad = False

    print("Model loaded")

    # Initialize RISE explainer
    explainer = RISE(model, (224, 224))
    N, s, p1 = 100, 8, 0.2
    explainer.generate_masks(N, s, p1)
    explainer.load_masks('masks.npy', p1, s)

    print("RISE explainer initialized")

    # Set up metrics
    deletion_metric = Deletion(data_replace_method="black", bound_max_step=True, batch_size=1, max_step_nb=14*14, cumulative=True)
    insertion_metric = Insertion(data_replace_method="blur", bound_max_step=True, batch_size=1, max_step_nb=14*14, cumulative=True)

    print(f"Checking image folder: {image_folder}")
    print(f"Folder exists: {os.path.exists(image_folder)}")
    print(f"Folder contents: {os.listdir(image_folder)}")

    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    print(f"Found {len(image_files)} images in the folder")
    print(f"Image files: {image_files}")

    # Function to calculate sparsity
    def calculate_sparsity(saliency_map):
        max_value = np.max(saliency_map)
        mean_value = np.mean(saliency_map)
        return max_value / mean_value if mean_value != 0 else float('inf')

    # Initialize dictionary to store results for each image
    image_results = {}

    # Process each image
    for image_file in tqdm(image_files, desc="Processing images"):
        img_path = os.path.join(image_folder, image_file)
        img = read_tensor(img_path).cuda()

        # Get prediction
        with torch.no_grad():
            output = model(img)
        target_class = output.argmax().item()

        # Generate saliency map
        sal_og = explainer(img)[target_class].cpu().numpy()

        # Calculate metrics
        saliency_map_tensor = torch.from_numpy(sal_og).float().unsqueeze(0).unsqueeze(0).cuda()

        # Compute Deletion metrics
        deletion_result = deletion_metric(model, img, saliency_map_tensor, class_to_explain_list=[target_class])

        # Compute Insertion metrics
        insertion_result = insertion_metric(model, img, saliency_map_tensor, class_to_explain_list=[target_class])

        # Calculate sparsity
        sparsity = calculate_sparsity(sal_og)

        # Store results for this image
        image_results[image_file] = {
            "DAUC": deletion_result["dauc"],
            "DC": deletion_result["dc"],
            "IAUC": insertion_result["iauc"],
            "IC": insertion_result["ic"],
            "Sparsity": sparsity
        }

        # Visualize and save saliency map
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        tensor_imshow(img[0].cpu())
        plt.title(f"Original - {get_class_name(target_class)}")
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        tensor_imshow(img[0].cpu())
        plt.imshow(sal_og, cmap='jet', alpha=0.5)
        plt.title("Saliency Map")
        plt.axis('off')
        
        plt.savefig(f"saliency_{image_file}.png")
        plt.close()

    # Print individual results if analysis is True
    if analysis:
        for image_file, results in image_results.items():
            print(f"\nMetrics for {image_file}:")
            
            headers = ["Metric", "Value"]
            table_data = [[key, f"{value:.4f}"] for key, value in results.items()]
            
            table = tabulate(table_data, headers=headers, tablefmt="grid")
            print(table)

    # Calculate overall averages and standard deviations
    overall_results = {key: [] for key in ["DAUC", "DC", "IAUC", "IC", "Sparsity"]}

    for results in image_results.values():
        for key, value in results.items():
            overall_results[key].append(value)

    # Create the overall summary table
    print("\nOverall Summary:")
    headers = ["Metric", "Avg ± Std"]
    summary_data = []

    for key, values in overall_results.items():
        avg = np.mean(values)
        std = np.std(values)
        summary_data.append([key, f"{avg:.4f} ± {std:.4f}"])

    summary_table = tabulate(summary_data, headers=headers, tablefmt="grid")
    print(summary_table)

    print("Analysis completed")

# Usage
image_folder = r'C:\Users\eagua\OneDrive\Desktop\data_benchmark'
analyze_saliency_maps(image_folder, analysis=True)  # Set analysis=False to skip individual results

Script started
Model loaded


Generating filters: 100%|██████████| 100/100 [00:00<00:00, 925.93it/s]


RISE explainer initialized
Checking image folder: C:\Users\eagua\OneDrive\Desktop\data_benchmark
Folder exists: True
Folder contents: ['.DS_Store', 'catdog.png', 'golden-retriever-puppy.jpg', 'good_evaluations', 'ILSVRC2012_val_00000110.JPEG', 'ILSVRC2012_val_00000111.JPEG', 'ILSVRC2012_val_00000113.JPEG', 'ILSVRC2012_val_00000119.JPEG', 'ILSVRC2012_val_00000124.JPEG', 'ILSVRC2012_val_00000128.JPEG', 'ILSVRC2012_val_00000133.JPEG', 'ILSVRC2012_val_00000134.JPEG', 'ILSVRC2012_val_00000139.JPEG', 'ILSVRC2012_val_00000140.JPEG', 'ILSVRC2012_val_00000142.JPEG', 'ILSVRC2012_val_00000148.JPEG', 'ILSVRC2012_val_00000204.JPEG', 'ILSVRC2012_val_00000212.JPEG', 'ILSVRC2012_val_00000219.JPEG', 'ILSVRC2012_val_00000248.JPEG', 'ILSVRC2012_val_00000269.JPEG', 'ILSVRC2012_val_00000282.JPEG', 'ILSVRC2012_val_00000285.JPEG', 'ILSVRC2012_val_00000286.JPEG', 'ILSVRC2012_val_00000608.JPEG', 'ILSVRC2012_val_00000614.JPEG', 'ILSVRC2012_val_00000615.JPEG', 'ILSVRC2012_val_00000617.JPEG', 'ILSVRC2012_val_0000

Processing images: 100%|██████████| 98/98 [03:02<00:00,  1.86s/it]


Metrics for catdog.png:
+----------+---------+
| Metric   |   Value |
+==========+=========+
| DAUC     |  0.0437 |
+----------+---------+
| DC       |  0.1405 |
+----------+---------+
| IAUC     |  0.3624 |
+----------+---------+
| IC       |  0.0451 |
+----------+---------+
| Sparsity |  2.5457 |
+----------+---------+

Metrics for golden-retriever-puppy.jpg:
+----------+---------+
| Metric   |   Value |
+==========+=========+
| DAUC     |  0.1306 |
+----------+---------+
| DC       |  0.091  |
+----------+---------+
| IAUC     |  0.1815 |
+----------+---------+
| IC       | -0.0548 |
+----------+---------+
| Sparsity |  2.1014 |
+----------+---------+

Metrics for ILSVRC2012_val_00000110.JPEG:
+----------+---------+
| Metric   |   Value |
+==========+=========+
| DAUC     |  0.021  |
+----------+---------+
| DC       |  0.0072 |
+----------+---------+
| IAUC     |  0.148  |
+----------+---------+
| IC       |  0.0336 |
+----------+---------+
| Sparsity |  3.1057 |
+----------+-----